In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# December 16 games

appalachian = MoneyLine(event="appalachian", bet_amount=100, odds=180)
salabama = MoneyLine(event="salabama", bet_amount=100, odds=-225) 

lakers = MoneyLine(event="lakers", bet_amount=100, odds=180) 
bucks = MoneyLine(event="bucks", bet_amount=100, odds=-225)

nets = MoneyLine(event="nets", bet_amount=100, odds=155)
spurs = MoneyLine(event="spurs", bet_amount=100, odds=-186)

rockets = MoneyLine(event="rockets", bet_amount=100, odds=175)
clippers = MoneyLine(event="clippers", bet_amount=100, odds=-210)

binaries = [
    [appalachian, salabama],
    [lakers, bucks],
#     [nets, spurs],
    [rockets, clippers]
]

results = [
    [0, 1], #[appalachian, salabama],
    [0, 1], #[lakers, bucks],
#     [0, 1], #[nets, spurs],
    [0, 1], #[rockets, clippers],
]

def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

1

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.30,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.1388972549980318
     jac: array([-1.82      , -0.57166667, -0.51527778,  0.1287037 , -0.51527778,
        0.1287037 ,  0.15779321,  0.49000588])
 message: 'Positive directional derivative for linesearch'
    nfev: 177
     nit: 15
    njev: 11
  status: 8
 success: False
       x: array([0.03561317, 0.0663439 , 0.06903477, 0.07351282, 0.06903477,
       0.07351282, 0.07076785, 0.01      ])
slsqp_solver: 
                         event    index[]     result  event_status   bet  \
7      salabama_bucks_clippers  [1, 3, 5]  [1, 1, 1]          True  0.01   
0   appalachian_lakers_rockets  [0, 2, 4]  [0, 0, 0]         False  0.04   
1  appalachian_lakers_clippers  [0, 2, 5]  [0, 0, 1]         False  0.07   
2    appalachian_bucks_rockets  [0, 3, 4]  [0, 1, 0]         False  0.07   
4      salabama_lakers_rockets  [1, 2, 4]  [1, 0, 0]         False  0.07   
3   appalachian_bucks_clippers  [0, 3, 5]  [0, 1, 1]         False  0.07   
5     salabama_lakers_clippers  [1, 2, 5]  [1,

In [8]:
df.describe()

,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000
mean,0.058750,9.517016,0.548225,0.080000
std,0.022321,5.797685,0.268518,0.269338
min,0.010000,3.079953,0.030800,-0.440000
25%,0.062500,5.912191,0.430675,-0.037500
50%,0.070000,8.546296,0.603350,0.135000
75%,0.070000,11.235000,0.767800,0.300000
max,0.070000,21.560000,0.767800,0.300000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,4.000,4.000000,4.000000,4.000000
mean,0.055,5.189587,0.328650,-0.140000
std,0.030,1.410695,0.199171,0.200499
min,0.010,3.079953,0.030800,-0.440000
25%,0.055,5.073229,0.312200,-0.155000
50%,0.070,5.854012,0.422450,-0.045000
75%,0.070,5.970370,0.438900,-0.030000
max,0.070,5.970370,0.438900,-0.030000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,4.0000,4.000000,4.0000,4.0
mean,0.0625,13.844444,0.7678,0.3
std,0.0150,5.148098,0.0000,0.0
min,0.0400,11.122222,0.7678,0.3
25%,0.0625,11.122222,0.7678,0.3
50%,0.0700,11.347778,0.7678,0.3
75%,0.0700,14.070000,0.7678,0.3
max,0.0700,21.560000,0.7678,0.3


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
7,salabama_bucks_clippers,"[1, 3, 5]","[1, 1, 1]",True,0.01,3.079953,0.0308,-0.44
6,salabama_bucks_rockets,"[1, 3, 4]","[1, 1, 0]",False,0.07,5.737654,0.4060,-0.06
3,appalachian_bucks_clippers,"[0, 3, 5]","[0, 1, 1]",False,0.07,5.970370,0.4389,-0.03
5,salabama_lakers_clippers,"[1, 2, 5]","[1, 0, 1]",False,0.07,5.970370,0.4389,-0.03


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
0,appalachian_lakers_rockets,"[0, 2, 4]","[0, 0, 0]",False,0.04,21.560000,0.7678,0.3
1,appalachian_lakers_clippers,"[0, 2, 5]","[0, 0, 1]",False,0.07,11.573333,0.7678,0.3
2,appalachian_bucks_rockets,"[0, 3, 4]","[0, 1, 0]",False,0.07,11.122222,0.7678,0.3
4,salabama_lakers_rockets,"[1, 2, 4]","[1, 0, 0]",False,0.07,11.122222,0.7678,0.3


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,4,4,4,4,4,4,4


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,3,3,3,3,3,3,3
True,1,1,1,1,1,1,1


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

def color_positive_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'green' if val == True else 'black'
    return 'color: %s' % color

In [17]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,salabama_bucks_clippers,"['1', '3', '5']","[1, 1, 1]",True,0.01,3.07995,0.0308,-0.44
1,salabama_bucks_rockets,"['1', '3', '4']","[1, 1, 0]",False,0.07,5.73765,0.406,-0.06
2,appalachian_bucks_clippers,"['0', '3', '5']","[0, 1, 1]",False,0.07,5.97037,0.4389,-0.03
3,salabama_lakers_clippers,"['1', '2', '5']","[1, 0, 1]",False,0.07,5.97037,0.4389,-0.03


In [18]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,appalachian_lakers_rockets,"['0', '2', '4']","[0, 0, 0]",False,0.04,21.56,0.7678,0.3
1,appalachian_lakers_clippers,"['0', '2', '5']","[0, 0, 1]",False,0.07,11.5733,0.7678,0.3
2,appalachian_bucks_rockets,"['0', '3', '4']","[0, 1, 0]",False,0.07,11.1222,0.7678,0.3
3,salabama_lakers_rockets,"['1', '2', '4']","[1, 0, 0]",False,0.07,11.1222,0.7678,0.3
